<a href="https://colab.research.google.com/github/Ramubala/hindi-tokenizer/blob/main/Tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>